In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [6]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [30]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thread")]
    return soup

In [31]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [32]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [49]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [53]:
base_cols = None
games = []

for box_scores in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 29
200 / 29
300 / 29
400 / 29
500 / 29
600 / 29
700 / 29
800 / 29
900 / 29
1000 / 29
1100 / 29
1200 / 29
1300 / 29
1400 / 29
1500 / 29
1600 / 29
1700 / 29
1800 / 29
1900 / 29
2000 / 29
2100 / 29
2200 / 29
2300 / 29
2400 / 29
2500 / 29
2600 / 29
2700 / 29
2800 / 29
2900 / 29
3000 / 29
3100 / 29
3200 / 29
3300 / 29
3400 / 29
3500 / 29
3600 / 29
3700 / 29
3800 / 29
3900 / 29
4000 / 29
4100 / 29
4200 / 29
4300 / 29
4400 / 29
4500 / 29
4600 / 29
4700 / 29
4800 / 29
4900 / 29
5000 / 29
5100 / 29
5200 / 29
5300 / 29
5400 / 29
5500 / 29
5600 / 29
5700 / 29
5800 / 29
5900 / 29
6000 / 29
6100 / 29
6200 / 29
6300 / 29
6400 / 29
6500 / 29
6600 / 29
6700 / 29
6800 / 29
6900 / 29
7000 / 29
7100 / 29
7200 / 29
7300 / 29
7400 / 29
7500 / 29
7600 / 29
7700 / 29
7800 / 29
7900 / 29
8000 / 29
8100 / 29
8200 / 29
8300 / 29
8400 / 29
8500 / 29
8600 / 29
8700 / 29
8800 / 29


In [55]:
games_df = pd.concat(games, ignore_index=True)

In [56]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
3,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
4,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
17768,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
17769,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
17770,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True


In [57]:
games_df.to_csv("nba_games.csv")